In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from tqdm import tqdm

## 데이터 불러오기

In [ ]:
raw_data=pd.read_csv('./data09/funda_train.csv')
data=raw_data.copy()
data.info()
#data.describe()  
# 날짜 헷갈리니까 합친다.
# 환불 금액이 보임. 환불을 먼저 없앤다.

## 환불금액제거.

In [ ]:
#합치기
data['date']=data['transacted_date']+' '+data['transacted_time']
data['date']=pd.to_datetime(data['date'])
#환불 제거
data_neg=data[data['amount']<0]
data_pos=data[data['amount']>0]
remove_data=pd.DataFrame()
# store_id 안나누고 하니까 시간이 너무 오래걸림. 
for i in tqdm(data.store_id.unique()):
    divided_data=data_pos[data_pos['store_id']==i] 
    divided_data2=data_neg[data_neg['store_id']==i] 
    #행 마다 조사.    
    for neg in divided_data2.to_records():  
        refund_store=neg['store_id']
        refund_id=neg['card_id'] 
        refund_date=neg['date']
        refund_amount=abs(neg['amount'])
        #행뽑기
        row=divided_data[divided_data['date']<=refund_date]
        row=row[row['card_id']==refund_id]
        row=row[row['amount']==refund_amount]
        
        if(len(row)!=0):
            refund_date=max(row['date']) 
            remove=divided_data[divided_data['date']==refund_date] 
            remove=remove[remove['card_id']==refund_id] 
            remove=remove[remove['amount']==refund_amount] 
            divided_data=divided_data.drop(index=remove.index) 

    remove_data=pd.concat([remove_data,divided_data],axis=0)

remove_data

#remove_data.to_csv('./data09/funda_remove_data.csv')

In [2]:
data=pd.read_csv('./data09/funda_remove_data.csv',index_col=0)
data['date_slice']=pd.to_datetime(data['date'].str.slice(stop=10))
data=data.set_index('date_slice')
#data.head(3)

C:\Users\ckdck\miniconda3\envs\tensor\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
data_month=pd.DataFrame()
for i in tqdm(data.store_id.unique()):
    data_num=data[data.store_id==i]
    sum_amount=data_num['amount'].resample(rule='m').sum()
    data_mon=pd.concat([sum_amount],axis=1)
    data_mon.insert(0,'store_id',i)
    data_month=pd.concat([data_month,data_mon],axis=0)
    
#data_month.head(10)

100%|██████████████████████████████████████████████████████████████████████████████| 1967/1967 [00:49<00:00, 39.71it/s]


## ARIMA

In [4]:
import itertools

p = [0,1,2]
d = [0,1,2]
q = [0,1,2]

pdq = list(itertools.product(p, d, q))

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

arima_pred_arr = np.array([])
for i in tqdm(data_month.store_id.unique()):
    data_set = data_month[data_month.store_id == i]

    best_score = 10000000000
    best_param = 0
    for param in pdq:
        try:
            arima_model = ARIMA(data_set.amount.values, order=param)
            result = arima_model.fit()
            if result.aic < best_score:
                best_score = result.aic
                best_param = param
        except:
            continue

    arima_model = ARIMA(data_set.amount.values, order=best_param)
    arima_result = arima_model.fit()
    arima_pred = arima_result.forecast(3)[0]

    arima_pred_arr = np.concatenate((arima_pred_arr, np.array([arima_pred.sum()])))

In [ ]:
arima_sum_sub = pd.DataFrame({'store_id' : data_month.store_id.unique().tolist(), 'amount' : arima_pred_arr})
arima_sum_sub.to_csv('./data09/funda_arima_sub.csv', index=False)